In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import tarfile
import json
import random
import shutil
import subprocess
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import warnings

import torch
import torch.nn as nn
import torchvision
import torchvision.models as models
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from pycocotools.coco import COCO
from PIL import Image
from itertools import cycle
from pathlib import Path


from ultralytics import YOLO
from torchmetrics.detection import MeanAveragePrecision
from torchmetrics.segmentation import MeanIoU
from torchmetrics.classification import Accuracy

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
print(torch.cuda.current_device())
print(torch.cuda.get_device_properties(0))
warnings.filterwarnings("ignore")

def set_seed(seed_value=42):
    """
    設定所有相關函式庫的隨機種子以確保實驗的可重現性。
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  # if you are using multi-GPU.
        
        # 這兩行是確保 CUDA 演算法是確定性的，對於可重現性至關重要
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False


d:\miniconda3\envs\god\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0
_CudaDeviceProperties(name='NVIDIA GeForce RTX 4060 Laptop GPU', major=8, minor=9, total_memory=8187MB, multi_processor_count=24, uuid=fc610937-b9ef-116c-0148-9424d82851c8, L2_cache_size=32MB)


## 資料集下載

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("awsaf49/coco-2017-dataset")

print("Path to dataset files:", path)

d:\miniconda3\envs\god\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 25.0G/25.0G [14:49<00:00, 30.2MB/s]  

Extracting files...


Path to dataset files: C:\Users\User\.cache\kagglehub\datasets\awsaf49\coco-2017-dataset\versions\2


In [ ]:
# === 路徑設定 ===
RAW_DIR = 'data/raw'
# MINI_DIR = 'data/mini_datasets'
# os.makedirs(RAW_DIR, exist_ok=True)
# os.makedirs(MINI_DIR, exist_ok=True)

# # === 1. 使用 Kaggle CLI 下載並解壓縮資料集 ===
datasets = {
    'mini_coco_det': 'awsaf49/coco-2017-dataset',
    # 'mini_voc_seg': 'gopalbhattrai/pascal-voc-2012-dataset',
    # 'imagenette': 'jhoward/imagenette-160-px'
}

for key, kaggle_id in datasets.items():
    # 下載 .zip
    subprocess.run([
        'kaggle', 'datasets', 'download',
        '-d', kaggle_id,
        '-p', RAW_DIR,
        '--force'
    ], check=True)
    
    # 解壓 .zip
    zip_name = kaggle_id.split('/')[-1] + '.zip'
    zip_path = os.path.join(RAW_DIR, zip_name)
    extract_to = os.path.join(RAW_DIR, key)
    os.makedirs(extract_to, exist_ok=True)
    shutil.unpack_archive(zip_path, extract_to)



In [ ]:
# 絕對路徑（請確認這是正確的）
tgz_path = r'D:\DL_HW\\HW2\data\\raw\\imagenette\\imagenette-160.tgz'
extract_to = r'D:\DL_HW\\HW2\data\\raw\\imagenette'

# 確認檔案存在
if os.path.exists(tgz_path):
    print(f"🔍 解壓 {tgz_path}")
    with tarfile.open(tgz_path, 'r:gz') as tar:
        tar.extractall(path=extract_to)
    print(f"✅ 解壓完成，資料已放到 {extract_to}")
else:
    print(f"❌ 找不到檔案：{tgz_path}")


🔍 解壓 D:\DL_HW\\HW2\data\\raw\\imagenette\\imagenette-160.tgz
✅ 解壓完成，資料已放到 D:\DL_HW\\HW2\data\\raw\\imagenette


In [7]:
raw_dir = 'raw'

print(f"🔍 掃描資料夾（僅資料夾，不含檔案）：{raw_dir}")

visited = set()

for root, dirs, files in os.walk(raw_dir):
    rel_root = os.path.relpath(root, raw_dir)
    if rel_root == '.':
        rel_root = ''
    level = rel_root.count(os.sep)
    indent = ' ' * 2 * level
    folder_name = os.path.basename(root)
    folder_path = os.path.normpath(root)
    if folder_path not in visited:
        print(f"{indent}📁 {folder_name}/")
        visited.add(folder_path)


🔍 掃描資料夾（僅資料夾，不含檔案）：raw
📁 raw/
📁 imagenette/
  📁 imagenette-160/
    📁 train/
      📁 n01440764/
      📁 n02102040/
      📁 n02979186/
      📁 n03000684/
      📁 n03028079/
      📁 n03394916/
      📁 n03417042/
      📁 n03425413/
      📁 n03445777/
      📁 n03888257/
    📁 val/
      📁 n01440764/
      📁 n02102040/
      📁 n02979186/
      📁 n03000684/
      📁 n03028079/
      📁 n03394916/
      📁 n03417042/
      📁 n03425413/
      📁 n03445777/
      📁 n03888257/
📁 mini_coco_det/
  📁 annotations/
  📁 test2017/
  📁 train2017/
  📁 val2017/
📁 mini_voc_seg/
  📁 VOC2012_test/
    📁 VOC2012_test/
      📁 Annotations/
      📁 ImageSets/
        📁 Action/
        📁 Layout/
        📁 Main/
        📁 Segmentation/
      📁 JPEGImages/
  📁 VOC2012_train_val/
    📁 VOC2012_train_val/
      📁 Annotations/
      📁 ImageSets/
        📁 Action/
        📁 Layout/
        📁 Main/
        📁 Segmentation/
      📁 JPEGImages/
      📁 SegmentationClass/
      📁 SegmentationObject/


## Sampling

In [2]:
def safe_mkdir(path):
    os.makedirs(path, exist_ok=True)

# 主資料夾
RAW_DIR = 'data/raw'
OUTPUT_DIR = 'data/'

In [4]:
# 1️⃣ Mini-COCO-Det
# 設定路徑
RAW_DIR = 'raw/mini_coco_det'
OUTPUT_DIR = r'D:\DL_HW\\HW2\data\\mini_coco_det'

train_img_dir = os.path.join(RAW_DIR, 'train2017')
val_img_dir = os.path.join(RAW_DIR, 'val2017')
ann_dir = os.path.join(RAW_DIR, 'annotations')

# 載入 mini annotations
with open(os.path.join(ann_dir, 'instances_train2017.json')) as f:
    train_ann = json.load(f)
with open(os.path.join(ann_dir, 'instances_val2017.json')) as f:
    val_ann = json.load(f)

def sample_and_save_coco(ann_data, image_dir, split, sample_size):
    images = ann_data['images']
    if sample_size > len(images):
        raise ValueError(f"❌ 要抽樣的數量 {sample_size} 超過總數 {len(images)}")
    
    sampled_images = random.sample(images, sample_size)
    sampled_ids = {img['id'] for img in sampled_images}
    sampled_ann = {
        'images': sampled_images,
        'annotations': [ann for ann in ann_data['annotations'] if ann['image_id'] in sampled_ids],
        'categories': ann_data['categories']
    }

    # 複製圖片到指定資料夾
    img_out_dir = os.path.join(OUTPUT_DIR, split, 'images')
    ann_out_dir = os.path.join(OUTPUT_DIR, split, 'annotations')
    os.makedirs(img_out_dir, exist_ok=True)
    os.makedirs(ann_out_dir, exist_ok=True)

    for img in sampled_images:
        src_path = os.path.join(image_dir, img['file_name'])
        dst_path = os.path.join(img_out_dir, img['file_name'])
        shutil.copy2(src_path, dst_path)

    # 儲存 annotations JSON
    ann_out_path = os.path.join(ann_out_dir, f'instances_{split}_mini.json')
    with open(ann_out_path, 'w') as f:
        json.dump(sampled_ann, f)
    print(f"✅ {split.upper()} mini annotations 已儲存至 {ann_out_path}")

# 處理 train → 抽 240
sample_and_save_coco(train_ann, train_img_dir, 'train', 240)

# 處理 val → 抽 60
sample_and_save_coco(val_ann, val_img_dir, 'val', 60)

print("✅ COCO 抽樣與複製完成")

✅ TRAIN mini annotations 已儲存至 D:\DL_HW\\HW2\data\\mini_coco_det\train\annotations\instances_train_mini.json
✅ VAL mini annotations 已儲存至 D:\DL_HW\\HW2\data\\mini_coco_det\val\annotations\instances_val_mini.json
✅ COCO 抽樣與複製完成


In [ ]:
# 2️⃣ Mini-VOC-Seg
RAW_DIR = 'raw/'
OUTPUT_DIR = r'D:\DL_HW\\HW2\data'

voc_src_img = os.path.join(RAW_DIR, 'mini_voc_seg', 'VOC2012_train_val', 'VOC2012_train_val', 'JPEGImages')
voc_src_mask = os.path.join(RAW_DIR, 'mini_voc_seg', 'VOC2012_train_val', 'VOC2012_train_val', 'SegmentationClass')
voc_out = os.path.join(OUTPUT_DIR, 'mini_voc_seg')

# 列出有 mask 的圖片
mask_files = [f for f in os.listdir(voc_src_mask) if f.lower().endswith('.png')]
base_names = [f.replace('.png', '') for f in mask_files]
all_voc_imgs = [f + '.jpg' for f in base_names if os.path.exists(os.path.join(voc_src_img, f + '.jpg'))]

# 檢查每對圖片和 mask 是否都存在
missing_images = [f + '.jpg' for f in base_names if not os.path.exists(os.path.join(voc_src_img, f + '.jpg'))]
missing_masks = [f.replace('.jpg', '.png') for f in os.listdir(voc_src_img)
                 if f.lower().endswith('.jpg') and not os.path.exists(os.path.join(voc_src_mask, f.replace('.jpg', '.png')))]

if missing_images:
    print(f"❌ 缺少對應圖片（mask 有但圖檔沒找到）：{missing_images}")
if missing_masks:
    print(f"❌ 缺少對應 mask（圖片有但 mask 沒找到）：{missing_masks}")

if len(all_voc_imgs) < 300:
    raise ValueError(f"❌ 有效 VOC 圖片不足 300 張（目前只有 {len(all_voc_imgs)} 張）")

print(f"✅ 總共找到 {len(all_voc_imgs)} 張有對應 mask 的圖片")

# 抽樣 300 張，分成 240 train / 60 val
sampled_voc = random.sample(all_voc_imgs, 300)
train_f, val_f = sampled_voc[:240], sampled_voc[240:]

for split, files in [('train', train_f), ('val', val_f)]:
    img_out = os.path.join(voc_out, split, 'images')
    mask_out = os.path.join(voc_out, split, 'masks')
    safe_mkdir(img_out)
    safe_mkdir(mask_out)
    for fname in files:
        shutil.copy2(os.path.join(voc_src_img, fname), os.path.join(img_out, fname))
        mname = fname.replace('.jpg', '.png')
        shutil.copy2(os.path.join(voc_src_mask, mname), os.path.join(mask_out, mname))

print("✅ VOC 資料抽樣與複製完成")



❌ 缺少對應 mask（圖片有但 mask 沒找到）：['2007_000027.png', '2007_000272.png', '2007_000423.png', '2007_000664.png', '2007_000807.png', '2007_001558.png', '2007_001583.png', '2007_001627.png', '2007_001667.png', '2007_001686.png', '2007_002079.png', '2007_002262.png', '2007_002344.png', '2007_003091.png', '2007_003104.png', '2007_003226.png', '2007_003329.png', '2007_003581.png', '2007_003659.png', '2007_003745.png', '2007_003747.png', '2007_003831.png', '2007_004000.png', '2007_004049.png', '2007_004092.png', '2007_004133.png', '2007_004197.png', '2007_004238.png', '2007_004265.png', '2007_004397.png', '2007_004454.png', '2007_004517.png', '2007_004713.png', '2007_004770.png', '2007_004795.png', '2007_004831.png', '2007_005019.png', '2007_005206.png', '2007_005310.png', '2007_005405.png', '2007_005425.png', '2007_005450.png', '2007_005527.png', '2007_005657.png', '2007_005691.png', '2007_005695.png', '2007_005748.png', '2007_005764.png', '2007_005896.png', '2007_005969.png', '2007_005971.png', '20

In [9]:
# 3️⃣ Imagenette-160 (每類別 train: 24, val: 6)
RAW_DIR = 'raw'
OUTPUT_DIR = 'data'

imagenette_src = os.path.join(RAW_DIR, 'imagenette', 'imagenette-160')
imagenette_out = os.path.join(OUTPUT_DIR, 'imagenette_160')

for split in ['train', 'val']:
    split_src = os.path.join(imagenette_src, split)
    split_out = os.path.join(imagenette_out, split)
    classes = [d for d in os.listdir(split_src) if os.path.isdir(os.path.join(split_src, d))]
    print(f"\n📁 處理 {split} 資料夾，找到類別: {classes}")

    for cls in classes:
        cls_src_dir = os.path.join(split_src, cls)
        cls_out_dir = os.path.join(split_out, cls)
        safe_mkdir(cls_out_dir)

        imgs = [f for f in os.listdir(cls_src_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        print(f" - 類別 {cls}: 找到 {len(imgs)} 張圖片")
        if len(imgs) == 0:
            raise ValueError(f"❌ 錯誤：類別 {cls} 在 {split} 下沒有找到圖片！")

        if split == 'train':
            sample_num = min(24, len(imgs))
        else:
            sample_num = min(6, len(imgs))
        sampled = random.sample(imgs, sample_num)

        for f in sampled:
            shutil.copy2(os.path.join(cls_src_dir, f), os.path.join(cls_out_dir, f))

print("✅ Imagenette 抽樣與複製完成")


📁 處理 train 資料夾，找到類別: ['n01440764', 'n02102040', 'n02979186', 'n03000684', 'n03028079', 'n03394916', 'n03417042', 'n03425413', 'n03445777', 'n03888257']
 - 類別 n01440764: 找到 1300 張圖片
 - 類別 n02102040: 找到 1300 張圖片
 - 類別 n02979186: 找到 1300 張圖片
 - 類別 n03000684: 找到 1194 張圖片
 - 類別 n03028079: 找到 1300 張圖片
 - 類別 n03394916: 找到 1300 張圖片
 - 類別 n03417042: 找到 1300 張圖片
 - 類別 n03425413: 找到 1300 張圖片
 - 類別 n03445777: 找到 1300 張圖片
 - 類別 n03888257: 找到 1300 張圖片

📁 處理 val 資料夾，找到類別: ['n01440764', 'n02102040', 'n02979186', 'n03000684', 'n03028079', 'n03394916', 'n03417042', 'n03425413', 'n03445777', 'n03888257']
 - 類別 n01440764: 找到 50 張圖片
 - 類別 n02102040: 找到 50 張圖片
 - 類別 n02979186: 找到 50 張圖片
 - 類別 n03000684: 找到 50 張圖片
 - 類別 n03028079: 找到 50 張圖片
 - 類別 n03394916: 找到 50 張圖片
 - 類別 n03417042: 找到 50 張圖片
 - 類別 n03425413: 找到 50 張圖片
 - 類別 n03445777: 找到 50 張圖片
 - 類別 n03888257: 找到 50 張圖片
✅ Imagenette 抽樣與複製完成


# 模型建立

In [2]:
print(f"🔍 掃描資料夾（僅資料夾，不含檔案）：{'data'}")

visited = set()

for root, dirs, files in os.walk('data'):
    rel_root = os.path.relpath(root, 'data')
    if rel_root == '.':
        rel_root = ''
    level = rel_root.count(os.sep)
    indent = ' ' * 2 * level
    folder_name = os.path.basename(root)
    folder_path = os.path.normpath(root)
    if folder_path not in visited:
        print(f"{indent}📁 {folder_name}/")
        visited.add(folder_path)


🔍 掃描資料夾（僅資料夾，不含檔案）：data
📁 data/
📁 imagenette_160/
  📁 train/
    📁 n01440764/
    📁 n02102040/
    📁 n02979186/
    📁 n03000684/
    📁 n03028079/
    📁 n03394916/
    📁 n03417042/
    📁 n03425413/
    📁 n03445777/
    📁 n03888257/
  📁 val/
    📁 n01440764/
    📁 n02102040/
    📁 n02979186/
    📁 n03000684/
    📁 n03028079/
    📁 n03394916/
    📁 n03417042/
    📁 n03425413/
    📁 n03445777/
    📁 n03888257/
📁 mini_coco_det/
  📁 train/
    📁 annotations/
    📁 images/
  📁 val/
    📁 annotations/
    📁 images/
📁 mini_voc_seg/
  📁 train/
    📁 images/
    📁 masks/
  📁 val/
    📁 images/
    📁 masks/


In [2]:
# --- 1. 模型定義 (Model Definition) - 使用 YOLOv8 Backbone ---

class UnifiedModel(nn.Module):
    def __init__(self, num_classes=10, num_det_classes=10, num_seg_classes=21):
        super(UnifiedModel, self).__init__()
        
        # --- 骨幹網路 (Backbone) ---
        # 載入預訓練的 YOLOv8 Nano 模型
        yolo_base = YOLO('yolov8n.pt') 
        
        # 提取 YOLOv8 的骨幹網路部分 (前10層，直到 SPPF 層結束)
        # 這部分負責從輸入圖片中提取深層特徵
        self.backbone = yolo_base.model.model[:10]
        
        # YOLOv8n 骨幹網路輸出特徵圖的 channel 數是 256
        backbone_out_channels = 256
        
        # --- 統一頭部 (Unified Head) ---
        # 根據作業要求，建立一個 2-3 層的共享頭部
        # 因為 backbone_out_channels 變了，所以這裡的 in_channels 也要跟著改
        self.unified_head = nn.Sequential(
            nn.Conv2d(backbone_out_channels, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU()
        )
        
        # --- 任務專用輸出層 (Task-specific Output Layers) ---
        # 這部分的結構和之前完全一樣，因為它們是接在 unified_head 後面
        
        # 1. 分類 (Classification)
        self.cls_output = nn.Linear(256, num_classes)
        
        # 2. 物件偵測 (Object Detection)
        self.det_output = nn.Conv2d(256, 5 + num_det_classes, kernel_size=1)
        
        # 3. 語意分割 (Semantic Segmentation)
        self.seg_output = nn.Conv2d(256, num_seg_classes, kernel_size=1)

    def forward(self, x):
        # x 的維度: (batch_size, 3, H, W), e.g., (16, 3, 224, 224)
        
        # YOLOv8 的 backbone 需要特定大小的輸入，通常是 640x640
        # 但由於卷積的特性，它也能處理不同大小的輸入，這裡我們先直接傳入
        # 如果遇到問題，可能需要 F.interpolate(x, size=640)
        features = self.backbone(x)
        # features 維度: (batch_size, 1024, H/32, W/32), e.g., (16, 1024, 7, 7)
        
        unified_features = self.unified_head(features)
        # unified_features 維度: (batch_size, 256, H/32, W/32)

        # 分類輸出 (與之前相同)
        cls_pooled = F.adaptive_avg_pool2d(unified_features, 1).view(unified_features.size(0), -1)
        cls_out = self.cls_output(cls_pooled)
        
        # 偵測輸出 (與之前相同)
        det_out = self.det_output(unified_features)
        
        # 分割輸出 (與之前相同)
        seg_out_small = self.seg_output(unified_features)
        seg_out = F.interpolate(seg_out_small, size=x.shape[2:], mode='bilinear', align_corners=False)

        return cls_out, det_out, seg_out
    
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def postprocess_detection(det_out, conf_threshold=0.001, nms_threshold=0.5, img_size=512):
    """
    將模型的網格輸出轉換為標準的 bounding box 格式，並執行 NMS。
    
    Args:
        det_out (torch.Tensor): 模型的偵測輸出，維度為 (B, 5 + num_classes, H, W)。
        conf_threshold (float): 物件信心度閾值。
        nms_threshold (float): NMS 的 IoU 閾值。
        img_size (int): 輸入圖片的尺寸，用於將正規化座標轉換回像素座標。

    Returns:
        list[dict]: 一個列表，每個元素是一個字典，代表一張圖片的預測結果。
                    格式: [{'boxes': tensor, 'scores': tensor, 'labels': tensor}]
    """
    # det_out 維度: [batch_size, 5 + num_classes, grid_h, grid_w]
    # 5 + num_classes = [x, y, w, h, obj_conf, class_1, class_2, ...]
    
    batch_size = det_out.shape[0]
    num_classes = det_out.shape[1] - 5
    grid_h, grid_w = det_out.shape[2], det_out.shape[3]
    
    # 將輸出從 (B, C, H, W) 轉換為 (B, H*W, C)，方便處理每個預測
    det_out = det_out.permute(0, 2, 3, 1).contiguous()
    det_out = det_out.view(batch_size, grid_h * grid_w, -1)
    
    # 創建網格座標，用於將相對座標轉換為絕對座標
    grid_y, grid_x = torch.meshgrid(torch.arange(grid_h), torch.arange(grid_w), indexing='ij')
    grid_xy = torch.stack((grid_x, grid_y), dim=-1).view(1, -1, 2).to(det_out.device)
    
    # --- 解碼預測 ---
    # 對座標和物件信心度應用 sigmoid
    det_out[..., :2] = torch.sigmoid(det_out[..., :2])   # x, y offsets
    det_out[..., 4] = torch.sigmoid(det_out[..., 4])    # Objectness confidence
    det_out[..., 5:] = torch.sigmoid(det_out[..., 5:])  # Class confidences

    # 將相對座標轉換為相對於整張圖片的座標 (0-1之間)
    # Box center
    box_xy = (det_out[..., :2] + grid_xy) / torch.tensor([grid_w, grid_h], device=det_out.device)    # Box dimensions
    # 【關鍵修正點】將 exp 改成 sigmoid，與訓練時的解碼方式保持一致！
    box_wh = torch.sigmoid(det_out[..., 2:4]) 
    
    # 將中心點、寬高 (cx, cy, w, h) 轉換為左上角、右下角 (x1, y1, x2, y2)
    box_xy1 = box_xy - box_wh / 2
    box_xy2 = box_xy + box_wh / 2
    boxes = torch.cat((box_xy1, box_xy2), dim=-1)
    
    # 計算最終信心度 (物件信心度 * 類別信心度)
    obj_conf = det_out[..., 4:5]
    class_conf, class_labels = torch.max(det_out[..., 5:], dim=-1, keepdim=True)
    final_conf = obj_conf * class_conf
    
    # 最終輸出列表
    output_list = []
    
    for i in range(batch_size):
        # 過濾掉信心度太低的預測
        mask = final_conf[i].squeeze() >= conf_threshold
        
        # 提取符合條件的預測
        batch_boxes = boxes[i][mask]
        batch_scores = final_conf[i][mask].flatten()
        batch_labels = class_labels[i][mask].flatten()
        
        if batch_boxes.shape[0] == 0:
            output_list.append({'boxes': torch.empty(0, 4), 'scores': torch.empty(0), 'labels': torch.empty(0, dtype=torch.long)})
            continue

        # 將座標從正規化 (0-1) 轉換為像素值
        batch_boxes *= img_size

        # 執行 NMS (Non-Maximum Suppression)
        # torchvision.ops.nms 需要每個類別分開做，但為簡化，這裡我們先對所有類別一起做
        # 更好的做法是進行 class-wise NMS，但會更複雜
        nms_indices = torchvision.ops.nms(batch_boxes, batch_scores, nms_threshold)
        
        # 根據 NMS 結果選取最終的預測
        final_boxes = batch_boxes[nms_indices]
        final_scores = batch_scores[nms_indices]
        final_labels = batch_labels[nms_indices]
        
        output_list.append({
            'boxes': final_boxes,
            'scores': final_scores,
            'labels': final_labels,
        })
        
    return output_list



In [3]:
# --- 2. EWC 持續學習 ---
class EWC:
    def __init__(self, model: nn.Module, device):
        self.model = model
        self.device = device
        self.params = {n: p for n, p in self.model.named_parameters() if p.requires_grad}
        self.means = {}
        self.precision_matrices = self._diag_fisher()

    def _diag_fisher(self):
        precision_matrices = {}
        for n, p in self.params.items():
            precision_matrices[n] = torch.zeros_like(p.data)
        return precision_matrices

    def update(self, dataloader):
        self.model.eval()
        for n, p in self.params.items():
            self.precision_matrices[n].data.zero_()

        for inputs, targets, task_ids in tqdm(dataloader, desc="Calculating Fisher Matrix"):
            inputs = inputs.to(self.device) # inputs 通常總是張量，可以直接移動
            task_id = task_ids[0].item()    # 假設批次中的 task_id 相同

            # 根據 task_id 條件式處理 targets 的設備轉移
            if task_id == 0:  # 分類任務
                targets = targets.to(self.device)
            elif task_id == 1:  # 物件偵測任務
                # targets 是字典列表，EWC 中簡化的 Fisher 損失計算並未使用 targets
                # 如果將來需要使用 targets 中的張量，則需要遍歷列表和字典來移動每個張量：
                # targets = [{k: v.to(self.device) for k, v in t.items()} for t in targets]
                # 目前的 EWC 物件偵測損失計算 (例如 MSE 或 sum of squares) 不直接使用 targets，所以可以暫時不移動
                pass
            elif task_id == 2:  # 語義分割任務
                targets = targets.to(self.device)
            # else: 可以為其他可能的 task_id 添加處理邏輯

            self.model.zero_grad()
            # 確保模型和輸入都在同一個設備上
            cls_out, det_out, seg_out = self.model(inputs) 

            loss = 0
            if task_id == 0: # 分類
                loss = F.cross_entropy(cls_out, targets) # targets 已在 device 上
            elif task_id == 1: # 物件偵測
                # det_out 應在 device 上，因為 model 和 inputs 都在 device 上
                loss = F.mse_loss(det_out, torch.zeros_like(det_out)) 
            elif task_id == 2: # 語義分割
                loss = F.cross_entropy(seg_out, targets.long(), ignore_index=255) # targets 已在 device 上

            if isinstance(loss, torch.Tensor) and loss.requires_grad:
                loss.backward()
            else:
                print(f"警告：任務 {task_id} 的損失不是需要梯度的張量。跳過 EWC 更新中的反向傳播。")


            for n, p in self.model.named_parameters():
                 if p.grad is not None and n in self.precision_matrices: # 確保參數在追蹤列表中
                    # 使用 len(dataloader.dataset) 或 len(dataloader) 進行標準化
                    # len(dataloader) 是批次數量，len(dataloader.dataset) 是樣本總數
                    # 通常 Fisher 矩陣是整個資料集的平均，所以用 len(dataloader.dataset) 更合適
                    if len(dataloader.dataset) > 0:
                        self.precision_matrices[n].data += p.grad.data.pow(2) / len(dataloader.dataset)
        
        self.means = {n: p.clone().detach() for n, p in self.params.items() if p.requires_grad}
        self.model.train()

    def penalty(self, model: nn.Module):
        loss = 0
        for n, p in model.named_parameters():
            if n in self.means:
                _loss = self.precision_matrices[n] * (p - self.means[n]).pow(2)
                loss += _loss.sum()
        return loss

In [4]:
# --- 3. Replay buffer 持續學習 ---

class StratifiedReplayBuffer:
    """分層抽樣的Replay Buffer，每個任務最多存儲10張圖片"""
    
    def __init__(self, max_samples_per_task=10):
        self.max_samples_per_task = max_samples_per_task
        self.buffers = {}  # task_id -> list of samples
        
    def add_samples(self, task_id, samples):
        """
        添加樣本到buffer
        samples: list of (image, target, task_id)
        """
        if task_id not in self.buffers:
            self.buffers[task_id] = []
            
        # 如果是分類任務，進行分層抽樣
        if task_id == 0:  # Classification task
            self._add_classification_samples(task_id, samples)
        else:
            # 對於detection和segmentation，隨機抽樣
            self._add_random_samples(task_id, samples)
    
    def _add_classification_samples(self, task_id, samples):
        """對分類任務進行分層抽樣"""
        # 按類別分組
        class_samples = {}
        for sample in samples:
            _, label, _ = sample
            label = label.item() if torch.is_tensor(label) else label
            if label not in class_samples:
                class_samples[label] = []
            class_samples[label].append(sample)
        
        # 每個類別最多選擇 max_samples_per_task // num_classes 個樣本
        num_classes = len(class_samples)
        samples_per_class = max(1, self.max_samples_per_task // num_classes)
        
        selected_samples = []
        for class_label, class_sample_list in class_samples.items():
            # 隨機選擇該類別的樣本
            num_to_select = min(samples_per_class, len(class_sample_list))
            selected = random.sample(class_sample_list, num_to_select)
            selected_samples.extend(selected)
        
        # 如果還有剩餘空間，隨機補充
        remaining_space = self.max_samples_per_task - len(selected_samples)
        if remaining_space > 0:
            all_remaining = [s for s in samples if s not in selected_samples]
            if all_remaining:
                additional = random.sample(
                    all_remaining, 
                    min(remaining_space, len(all_remaining))
                )
                selected_samples.extend(additional)
        
        self.buffers[task_id] = selected_samples[:self.max_samples_per_task]
    
    def _add_random_samples(self, task_id, samples):
        """隨機抽樣"""
        num_to_select = min(self.max_samples_per_task, len(samples))
        selected_samples = random.sample(samples, num_to_select)
        self.buffers[task_id] = selected_samples
    
    def get_replay_samples(self, exclude_task=None):
        """獲取replay樣本"""
        replay_samples = []
        for task_id, samples in self.buffers.items():
            if exclude_task is None or task_id != exclude_task:
                replay_samples.extend(samples)
        return replay_samples
    
    def replay_collate(self, batch):
        imgs, targets, task_ids = zip(*batch)
        imgs = torch.stack(imgs)                     # (B, C, H, W)
        return imgs, list(targets), torch.tensor(task_ids)
    
    def create_replay_loader(self, batch_size=8, exclude_task=None):
        """創建replay data loader"""
        replay_samples = self.get_replay_samples(exclude_task)
        if not replay_samples:
            return None
            
        # 創建dataset
        replay_dataset = ReplayDataset(replay_samples)
        return DataLoader(replay_dataset, batch_size=batch_size, shuffle=True, collate_fn=self.replay_collate)
    

class ReplayDataset(Dataset):
    def __init__(self, samples):
        self.samples = samples
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        return self.samples[idx]


In [5]:
# --- 4. 損失函數 (Loss Functions) ---
def bbox_ciou(pred, target, eps=1e-7):
    """pred/target: (...,4) in  cx,cy,w,h  normalized to 0-1"""
    # 轉成左上右下
    p_x1, p_y1 = pred[...,0] - pred[...,2]/2, pred[...,1] - pred[...,3]/2
    p_x2, p_y2 = pred[...,0] + pred[...,2]/2, pred[...,1] + pred[...,3]/2
    t_x1, t_y1 = target[...,0] - target[...,2]/2, target[...,1] - target[...,3]/2
    t_x2, t_y2 = target[...,0] + target[...,2]/2, target[...,1] + target[...,3]/2
    # 交集
    inter_x1, inter_y1 = torch.max(p_x1,t_x1), torch.max(p_y1,t_y1)
    inter_x2, inter_y2 = torch.min(p_x2,t_x2), torch.min(p_y2,t_y2)
    inter = (inter_x2-inter_x1).clamp(0) * (inter_y2-inter_y1).clamp(0)
    # 面積
    area_p = (p_x2-p_x1).clamp(0) * (p_y2-p_y1).clamp(0) + eps
    area_t = (t_x2-t_x1).clamp(0) * (t_y2-t_y1).clamp(0) + eps
    union = area_p + area_t - inter + eps
    iou   = inter / union

    # 中心距
    center_dist = (pred[...,0]-target[...,0])**2 + (pred[...,1]-target[...,1])**2
    # 封閉盒對角線長
    c_x1, c_y1 = torch.min(p_x1,t_x1), torch.min(p_y1,t_y1)
    c_x2, c_y2 = torch.max(p_x2,t_x2), torch.max(p_y2,t_y2)
    c2 = (c_x2-c_x1)**2 + (c_y2-c_y1)**2 + eps

    # 寬高比例項
    v = (4/np.pi**2) * torch.pow(torch.atan(target[...,2]/(target[...,3]+eps))
                                 - torch.atan(pred[...,2]/(pred[...,3]+eps)),2)
    with torch.no_grad():
        alpha = v / (1-iou+v)
    ciou = iou - center_dist/c2 - alpha*v
    return 1-ciou  # 1-CIoU -> 可當 loss

def detection_loss(pred, targets, img_size=512, num_det_classes=10, λ_conf=1.0, λ_bbox=5.0, λ_cls=1.0):
    """
    計算物件偵測的損失。
    pred: (B, 5+num_det_classes, H, W) 模型的原始輸出
    targets: list[dict]，每張圖片一個字典
    """
    device = pred.device
    B, C, H, W = pred.shape
    pred = pred.permute(0, 2, 3, 1)  # 轉換維度至 B,H,W,C

    # 1. 從模型輸出中分離出不同部分的預測值
    obj_logit = pred[..., 4]
    cls_logit = pred[..., 5:]
    
    # 【修正點】對模型的原始 bbox 輸出進行解碼，使其與 GT 在相同尺度 (0-1)
    bbox_pred_decoded = torch.sigmoid(pred[..., :4]) # (cx,cy,w,h)

    total_loss = pred.new_zeros(())

    # 遍歷 batch 中的每一張圖片
    for b in range(B):
        t = targets[b]  # 當前圖片的 ground truth

        # 2. 【重要邏輯】處理沒有標註物件的圖片
        if len(t["boxes"]) == 0:
            # 如果沒有物件，這張圖全部都是背景
            # 只計算 objectness loss，目標是讓所有網格的信心度都趨近於 0
            bg_target = torch.zeros((H, W), device=device)
            total_loss += λ_conf * F.binary_cross_entropy_with_logits(obj_logit[b], bg_target)
            
            # 使用 continue 跳過這張圖片的後續處理，直接進入下一輪迴圈
            continue

        # 3. 【變數定義】只有當圖片確定有物件時，才建立對應的 target tensors
        tgt_obj = torch.zeros((H, W), device=device)
        tgt_bbox = torch.zeros((H, W, 4), device=device)
        tgt_cls = torch.zeros((H, W, num_det_classes), device=device)

        # 將 ground truth box 資訊填入 target tensors
        boxes = t["boxes"] / img_size  # 將像素座標標準化到 0-1
        cx, cy = (boxes[:, 0] + boxes[:, 2]) / 2, (boxes[:, 1] + boxes[:, 3]) / 2
        gw, gh = boxes[:, 2] - boxes[:, 0], boxes[:, 3] - boxes[:, 1]
        gx, gy = (cx * W).long().clamp(0, W - 1), (cy * H).long().clamp(0, H - 1)

        for i, (ix, iy) in enumerate(zip(gx, gy)):
            if iy < H and ix < W:
                tgt_obj[iy, ix] = 1
                tgt_bbox[iy, ix] = torch.tensor([cx[i], cy[i], gw[i], gh[i]], device=device)
                cls_id = int(t["labels"][i])
                if cls_id < num_det_classes:
                    tgt_cls[iy, ix, cls_id] = 1

        # 4. 計算這張圖片的各項損失
        #    只在有物件的網格 (positive mask) 上計算 bbox 和 class loss
        pos_mask = tgt_obj.bool()
        
        obj_loss = F.binary_cross_entropy_with_logits(obj_logit[b], tgt_obj)
        
        if pos_mask.any():
            # 使用解碼後的 bbox_pred_decoded 來計算損失
            bbox_loss = bbox_ciou(bbox_pred_decoded[b][pos_mask], tgt_bbox[pos_mask]).mean()
            # print(f"Batch {b}, bbox_loss: {bbox_loss.item()}") # <-- 暫時加入這行
            cls_loss = F.binary_cross_entropy_with_logits(cls_logit[b][pos_mask], tgt_cls[pos_mask])
        else:
            # 理論上因為有 continue，不會執行到這裡，但作為防呆措施
            bbox_loss = cls_loss = torch.zeros((), device=device)

        # 將各項損失加權後累加
        total_loss += λ_conf * obj_loss + λ_bbox * bbox_loss + λ_cls * cls_loss

    # 回傳整個 batch 的平均損失
    return total_loss / B

def segmentation_loss(pred, target):
    # pred: (batch, num_seg_classes, H, W)
    # target: (batch, H, W)
    return F.cross_entropy(pred, target.long(), ignore_index=255)

def classification_loss(pred, target):
    return F.cross_entropy(pred, target)

In [6]:
# --- 5. 資料集 (Dataset) ---

# 通用影像轉換
common_image_transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# VOC 語義分割遮罩的轉換
voc_mask_transform = transforms.Compose([
    transforms.Resize((512, 512), interpolation=transforms.InterpolationMode.NEAREST),
    # transforms.PILToTensor(), # 對於遮罩，使用 ToTensor() 更好地處理不同的 PIL 模式
])

class ImagenetteDataset(Dataset):
    def __init__(self, root_dir, transform=None, split='train'): # 為清晰起見添加了 split 參數，以備後用
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []
        self.class_to_idx = {}
        self.task_id = 0  # 分類任務

        class_folders = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
        for i, class_name in enumerate(class_folders):
            self.class_to_idx[class_name] = i
            class_path = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_path):
                if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                    self.samples.append((os.path.join(class_path, img_name), i))
        
        if not self.samples:
            print(f"警告：在 ImagenetteDataset 的路徑 {root_dir} 中找不到任何影像。")


    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(label, dtype=torch.long), torch.tensor(self.task_id, dtype=torch.long)

class COCODataset(Dataset):
    def __init__(self, img_root, ann_file, transform=None):
        self.img_root = img_root
        self.transform = transform
        self.coco = COCO(ann_file)
        self.ids = list(sorted(self.coco.imgs.keys()))
        self.task_id = 1  # 物件偵測任務

        SELECTED_CAT_IDS = [
            15,  # bird     ← 對應 fish
            18,  # dog
            75,  # remote   ← 對應 CD-player
            44,  # scissors ← chainsaw
            14,  # bench    ← castle
            74,  # keyboard ← French-horn
            8,   # truck    ← garbage-truck
            20,  # parking-meter ← gas-pump
            37,  # sports-ball   ← golf-ball
            38,  # kite          ← parachute
        ]

        # 建立從 COCO 類別 ID 到連續 0-9 範圍的映射
        # 假設標註檔案中的 10 個類別就是要使用的類別。
        self.cat_ids = sorted(self.coco.getCatIds())
        selected_in_file = [cid for cid in SELECTED_CAT_IDS if cid in self.cat_ids]

        if len(selected_in_file) == 0:
            raise ValueError(
                f"在標註檔 {ann_file} 找不到任何指定的 10 類別 "
                f"(期望 {SELECTED_CAT_IDS})，請檢查 JSON。"
            )

        # 建立「原始 id → 連續 0‥9」映射；若只找到部分類別，也依序編到 0,1,2…
        self.cat_id_to_continuous_id = {
            cid: idx for idx, cid in enumerate(selected_in_file)
        }

        # 給個友善提示，確認到底抓到了幾類
        if len(selected_in_file) < 10:
            print(
                f"⚠️  JSON 內僅發現 {len(selected_in_file)} / 10 類：{selected_in_file}；"
                "後續模型與度量仍會以實際找到的類別數為準。"
            )
        else:
            print("✅ 已對齊 10 類，映射如下：", self.cat_id_to_continuous_id)

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        img_info = self.coco.loadImgs(img_id)[0]
        img_path = os.path.join(self.img_root, img_info['file_name'])
        image = Image.open(img_path).convert('RGB')

        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids)

        boxes = []
        labels = []
        for ann in anns:
            # 確保類別在我們的映射中
            if ann['iscrowd'] == 0 and ann['category_id'] in self.cat_id_to_continuous_id : 
                xmin, ymin, w, h = ann['bbox']
                boxes.append([xmin, ymin, xmin + w, ymin + h])
                labels.append(self.cat_id_to_continuous_id[ann['category_id']])
        
        target = {}
        if boxes:
            target['boxes'] = torch.tensor(boxes, dtype=torch.float32)
            target['labels'] = torch.tensor(labels, dtype=torch.long)
        else: # 處理此類別集合中沒有標註的影像
            target['boxes'] = torch.empty((0, 4), dtype=torch.float32)
            target['labels'] = torch.empty((0,), dtype=torch.long)

        if self.transform:
            # 注意：像 Normalize 這樣的轉換不應應用於邊界框座標。
            # common_transform 用於影像。如果邊界框需要隨著影像縮放而調整，通常會分開處理。
            # 此處假設邊界框是絕對座標，且模型能處理各種輸入尺寸，或根據原始影像維度進行後處理。
            # 為簡單起見，如果影像被縮放到 224x224，邊界框理想情況下應進行縮放。
            # 然而，YOLO 風格的模型通常在內部處理此問題，或期望標準化到 0-1 之間相對於網格的座標。
            # 考慮到模型結構，我們假設影像轉換是主要的。
            # 對於 torchmetrics，除非另有說明，否則邊界框應為原始影像座標。
            # 如果應用了 Resize((224,224))，則應縮放邊界框。
            original_w, original_h = image.size
            image = self.transform(image) # 這已包含 ToTensor 和 Resize
            
            if 'boxes' in target and target['boxes'].numel() > 0:
                boxes_tensor = target['boxes']
                # 縮放邊界框：x_new = x_old * (new_dim / old_dim)
                boxes_tensor[:, 0] *= (512 / original_w)
                boxes_tensor[:, 1] *= (512 / original_h)
                boxes_tensor[:, 2] *= (512 / original_w)
                boxes_tensor[:, 3] *= (512 / original_h)
                target['boxes'] = boxes_tensor

        return image, target, torch.tensor(self.task_id, dtype=torch.long)

class VOCDataset(Dataset):
    def __init__(self, image_dir, mask_dir, image_transform=None, mask_transform_pil=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.image_transform = image_transform
        self.mask_transform_pil = mask_transform_pil # 用於 PIL 到 PIL 的轉換，如 Resize
        self.task_id = 2  # 語義分割任務

        self.image_filenames = sorted([f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        
        # 檢查對應的遮罩，並在必要時篩選影像
        valid_samples = []
        for img_name in self.image_filenames:
            mask_name = os.path.splitext(img_name)[0] + '.png'
            if os.path.exists(os.path.join(mask_dir, mask_name)):
                valid_samples.append(img_name)
            else:
                print(f"警告：在 VOCDataset 中找不到影像 {img_name} 的遮罩。將跳過此影像。")
        self.image_filenames = valid_samples

        if not self.image_filenames:
            print(f"警告：在 VOCDataset (影像路徑: {image_dir}, 遮罩路徑: {mask_dir}) 中找不到有效的影像-遮罩對。")

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_name = self.image_filenames[idx]
        img_path = os.path.join(self.image_dir, img_name)
        mask_name = os.path.splitext(img_name)[0] + '.png'
        mask_path = os.path.join(self.mask_dir, mask_name)

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path) # VOC 遮罩通常是 'P' (調色盤) 或 'L' 模式

        # 首先對影像和遮罩應用基於 PIL 的轉換 (例如 Resize)
        if self.mask_transform_pil: # 這應包含遮罩的 Resize
            mask = self.mask_transform_pil(mask)

        # 將遮罩轉換為 numpy 陣列，然後轉換為張量以保留整數標籤
        mask_np = np.array(mask, dtype=np.uint8) # 確保是 uint8
        mask_tensor = torch.from_numpy(mask_np).long() # 轉換為 LongTensor

        if self.image_transform: # 這包含影像的 ToTensor 和 Normalize
            image = self.image_transform(image)
        else: # 如果未指定其他轉換，則進行基本的 ToTensor
            image = transforms.ToTensor()(image)
            
        return image, mask_tensor, torch.tensor(self.task_id, dtype=torch.long)

In [7]:
# --- 6. 訓練與驗證 (Train & Validate) ---
def stage_train(model, task_key, train_loader, val_loaders,
                task_id, replay_buffer, device, epochs=100, ewc=None, λ_ewc=1000, λ_bbox=5.0, lr=1e-4):
    optimiser = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    # 追蹤當前任務的最佳分數
    best_score = -1.0
    # 建立一個映射，將 task_key 對應到它在 results 字典中的指標名稱
    metric_map = {
        'seg': ('seg_mIoU', 'seg_mIoU'),
        'det': ('det_mAP', 'map'),
        'cls': ('cls_top1_acc', 'cls_top1_acc')
    }
    metric_key, primary_metric = metric_map[task_key]

    for ep in range(epochs):
        model.train()
        for imgs, targets, _ in train_loader:
            imgs = imgs.to(device)
            # === forward ===
            cls_out, det_out, seg_out = model(imgs)

            if task_key=='cls':
                loss = classification_loss(cls_out, targets.to(device))
            elif task_key=='det':
                loss = detection_loss(det_out, targets, img_size=512, λ_bbox=λ_bbox)
            else:
                loss = segmentation_loss(seg_out, targets.to(device))

            # --- EWC ---
            if ewc:
                loss += λ_ewc * ewc.penalty(model)

            optimiser.zero_grad()
            loss.backward()
            optimiser.step()

            # --- 收集樣本（只第一次 epoch）---
            if ep==0:
                for i in range(len(imgs)):
                    sample_tgt = targets[i] if task_key!='cls' else targets[i].item()
                    replay_buffer.add_samples(task_id, [(imgs[i].cpu(), sample_tgt, task_id)])

        # === Replay step ===
        replay_loader = replay_buffer.create_replay_loader(batch_size=8,
                                                           exclude_task=task_id)
        if replay_loader:
            for r_imgs, r_targets, r_task_ids in replay_loader:
                r_imgs = r_imgs.to(device)
                cls_out, det_out, seg_out = model(r_imgs)
                batch_loss = 0.
                for i in range(len(r_imgs)):
                    if r_task_ids[i]==0:
                        batch_loss += classification_loss(cls_out[i:i+1], torch.tensor([r_targets[i]]).to(device))
                    elif r_task_ids[i]==1:
                        batch_loss += detection_loss(det_out[i:i+1], [r_targets[i]], img_size=512, λ_bbox=10.0)
                    else:
                        batch_loss += segmentation_loss(seg_out[i:i+1], r_targets[i].unsqueeze(0).to(device))
                (batch_loss/len(r_imgs)).backward()
                optimiser.step()

        # --- 驗證與儲存檢查點 ---
        if (ep + 1) % 2 == 0:  # 您可以自行決定驗證的頻率
            val_results = validate(model, val_loaders, device)
            
            # 根據當前主要任務，取得對應的分數
            if task_key == 'det':
                current_score = val_results[metric_key][primary_metric]
            else:
                current_score = val_results[metric_key]

            # 檢查當前模型是否為此階段最佳
            if current_score > best_score:
                best_score = current_score
                
                # --- 這就是儲存的核心邏輯 ---
                checkpoint = {
                    'epoch': ep + 1,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimiser.state_dict(),
                    'best_score': best_score,
                    'task_key': task_key
                }
                
                # 儲存檢查點檔案
                save_path = f'checkpoint_{task_key}_best.pt'
                torch.save(checkpoint, save_path)
                print(f"✅ 任務 '{task_key}' 的新最佳模型已儲存至 {save_path}，分數為: {best_score:.4f}")

def validate(model, val_loaders, device):
    """對所有驗證集進行評估"""
    model.eval()
    
    # 初始化評估指標
    metrics = {
        'cls': Accuracy(task="multiclass", num_classes=10).to(device),
        'seg': MeanIoU(num_classes=21).to(device),
        'det': MeanAveragePrecision(box_format='xyxy', iou_type='bbox').to('cpu')
    }
    
    results = {}

    with torch.no_grad():

        # 1. 驗證分割任務
        for inputs, targets, _ in tqdm(val_loaders['seg'], desc="Validating SEG"):
            inputs, targets = inputs.to(device), targets.to(device)
            _, _, seg_out = model(inputs)
            preds_indices = torch.argmax(seg_out, dim=1)            # 手動將 preds 從 logits 轉換為類別索引
            metrics['seg'].update(preds_indices, targets.long())
        results['seg_mIoU'] = metrics['seg'].compute().item()
        metrics['seg'].reset()
        
        # 2. 驗證偵測任務
        # 你的 DataLoader 使用了 collate_fn，所以 target 會是一個 tuple of dicts
        # 我們需要將它轉換為 list of dicts
        for inputs, targets, _ in tqdm(val_loaders['det'], desc="Validating DET"):
            inputs = inputs.to(device)

            with torch.no_grad():
                _, det_out, _ = model(inputs)

            preds = postprocess_detection(det_out)  # list[dict]，仍在 GPU

            # ① 把 preds / targets 統一搬到 Metric 所在裝置 (這裡是 CPU)
            preds_cpu = [{k: v.cpu() for k, v in p.items()} for p in preds]
            targets_cpu = [
                {
                    'boxes':  t['boxes'].cpu(),
                    'labels': t['labels'].cpu()
                } for t in targets
            ]

            # ② 更新 mAP Metric
            metrics['det'].update(preds_cpu, targets_cpu)
            
        # 計算最終 mAP 結果
        # .compute() 會返回一個包含多個指標的字典
        det_map_results = metrics['det'].compute()

        # 修改字典推導式以處理可能的非純量張量
        processed_det_mAP = {}
        for k, v in det_map_results.items(): # 使用 det_map_results
            if isinstance(v, torch.Tensor) and v.numel() == 1:
                processed_det_mAP[k] = round(v.item(), 4)
            elif isinstance(v, (int, float)): # 有些指標可能直接是 float (例如 -1.0)
                processed_det_mAP[k] = round(v, 4)
            # else:
                # 可以選擇性地處理多元素張量，例如轉換為列表或忽略
                # print(f"指標 '{k}' 包含多個元素 ({v.shape if isinstance(v, torch.Tensor) else type(v)})，將不使用 .item()")
                # processed_det_mAP[k] = v.tolist() # 如果想保留它作為列表

        results['det_mAP'] = processed_det_mAP
        metrics['det'].reset()

        # 3. 驗證分類任務
        for inputs, targets, _ in tqdm(val_loaders['cls'], desc="Validating CLS"):
            inputs, targets = inputs.to(device), targets.to(device)
            cls_out, _, _ = model(inputs)
            metrics['cls'].update(cls_out, targets)
        results['cls_top1_acc'] = metrics['cls'].compute().item()
        metrics['cls'].reset()
        
    print(f"Validation Results -> Top-1 Acc: {results['cls_top1_acc']:.4f} | mIoU: {results['seg_mIoU']:.4f} | mAP: {results.get('det_mAP', 'N/A')}")
    return results

In [ ]:
# --- 7. 主程式 (Main Function) ---
def main():
    set_seed(7777777)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"使用裝置：{device}")
    batch_size = 6
    voc_mask_pil_resize_transform = transforms.Resize((512, 512), interpolation=transforms.InterpolationMode.NEAREST)

    # --- 為每個任務分別建立 Dataset 和 DataLoader ---
    data_root_dir = './data'
    train_cls_dataset = ImagenetteDataset(
        root_dir=os.path.join(data_root_dir, 'imagenette_160/train'),
        transform=common_image_transform,
        split='train'
    )
    val_cls_dataset = ImagenetteDataset(
        root_dir=os.path.join(data_root_dir, 'imagenette_160/val'),
        transform=common_image_transform,
        split='val'
    )
    train_det_dataset = COCODataset(
        img_root=os.path.join(data_root_dir, 'mini_coco_det/train/images'),
        ann_file=os.path.join(data_root_dir, 'mini_coco_det/train/annotations/instances_train_mini.json'),
        transform=common_image_transform
    )
    val_det_dataset = COCODataset(
        img_root=os.path.join(data_root_dir, 'mini_coco_det/val/images'),
        ann_file=os.path.join(data_root_dir, 'mini_coco_det/val/annotations/instances_val_mini.json'),
        transform=common_image_transform
    )
    train_seg_dataset = VOCDataset(
        image_dir=os.path.join(data_root_dir, 'mini_voc_seg/train/images'),
        mask_dir=os.path.join(data_root_dir, 'mini_voc_seg/train/masks'),
        image_transform=common_image_transform,
        mask_transform_pil=voc_mask_pil_resize_transform
    )
    val_seg_dataset = VOCDataset(
        image_dir=os.path.join(data_root_dir, 'mini_voc_seg/val/images'),
        mask_dir=os.path.join(data_root_dir, 'mini_voc_seg/val/masks'),
        image_transform=common_image_transform,
        mask_transform_pil=voc_mask_pil_resize_transform
    )

    def det_collate_fn(batch):
        images = torch.stack([item[0] for item in batch])
        targets = [item[1] for item in batch]
        task_ids = torch.stack([item[2] for item in batch])
        return images, targets, task_ids

    train_loaders = {
        'cls': DataLoader(train_cls_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True),
        'det': DataLoader(train_det_dataset, batch_size=batch_size, shuffle=True, collate_fn=det_collate_fn, num_workers=0, pin_memory=True),
        'seg': DataLoader(train_seg_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
    }
    val_loaders = {
        'cls': DataLoader(val_cls_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True),
        'det': DataLoader(val_det_dataset, batch_size=batch_size, shuffle=False, collate_fn=det_collate_fn, num_workers=0, pin_memory=True),
        'seg': DataLoader(val_seg_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
    }
    
    # --- 確認 Dataset 長度 ---
    print(f"訓練分類資料集樣本數：{len(train_cls_dataset)}")
    print(f"驗證分類資料集樣本數：{len(val_cls_dataset)}")
    print(f"訓練物件偵測資料集樣本數：{len(train_det_dataset)}")
    print(f"驗證物件偵測資料集樣本數：{len(val_det_dataset)}")
    print(f"訓練語義分割資料集樣本數：{len(train_seg_dataset)}")
    print(f"驗證語義分割資料集樣本數：{len(val_seg_dataset)}")

    # --- 初始化模型與持續學習工具 ---
    model = UnifiedModel(num_classes=10, num_det_classes=10, num_seg_classes=21).to(device)
    replay_buffer = StratifiedReplayBuffer(max_samples_per_task=10)
    
    # 【修改點 1】初始化 EWC 相關變數
    ewc = None
    ewc_lambda = 180000.0  # EWC 懲罰權重 (這是一個很重要的超參數，需要實驗調整)
    
    # 【修改點 2】為不同任務設定不同的訓練週期
    detection_epochs = 120
    classification_epochs = 30
    segmentation_epochs = 30

    metrics_log = {}

    # ========================== Stage 1: Segmentation ==========================
    print('🟢 Stage 1 — Segmentation')
    # 第一階段不需 EWC，因為還沒有舊任務
    stage_train(model, 'seg', train_loaders['seg'], val_loaders,
                task_id=2, replay_buffer=replay_buffer, device=device, epochs=segmentation_epochs, ewc=None)
    
    res = validate(model, val_loaders, device)
    metrics_log['seg_baseline'] = res['seg_mIoU']
    metrics_log['det_baseline'] = res['det_mAP']['map']
    metrics_log['cls_baseline'] = res['cls_top1_acc']
    
    # 【修改點 3】訓練完第一個任務後，計算其權重重要性
    print("Calculating Fisher Matrix for Segmentation Task...")
    ewc = EWC(model, device)
    ewc.update(train_loaders['seg'])
    
    # ========================== Stage 2: Detection ==========================
    print('🟡 Stage 2 — Detection')
    # 第二階段訓練時，傳入 ewc 物件來保護分割任務的權重
    stage_train(model, 'det', train_loaders['det'], val_loaders,
                task_id=1, replay_buffer=replay_buffer, device=device, epochs=detection_epochs,
                ewc=ewc, λ_ewc=ewc_lambda)  # <-- 啟用 EWC，並增加 epochs
    
    res = validate(model, val_loaders, device)
    metrics_log['seg_after_det'] = res['seg_mIoU']
    metrics_log['det_after_det'] = res['det_mAP']['map']
    metrics_log['cls_after_det'] = res['cls_top1_acc']

    # 【修改點 4】再次更新 EWC，疊加偵測任務的權重重要性
    print("Updating Fisher Matrix for Detection Task...")
    ewc.update(train_loaders['det'])

    # ========================== Stage 3: Classification ==========================
    print('🔵 Stage 3 — Classification')
    # 第三階段訓練時，傳入的 ewc 物件已包含前兩個任務的權重保護資訊
    stage_train(model, 'cls', train_loaders['cls'], val_loaders,
                task_id=0, replay_buffer=replay_buffer, device=device, epochs=classification_epochs,
                ewc=ewc, λ_ewc=ewc_lambda, lr=5e-5)  # <-- 啟用 EWC
    
    res = validate(model, val_loaders, device)
    metrics_log['seg_final'] = res['seg_mIoU']
    metrics_log['det_final'] = res['det_mAP']['map']
    metrics_log['cls_final'] = res['cls_top1_acc']
    
    # ========================== 產出結果表格 ==========================
    def pct_drop(after, base):
        return 100. * (base - after) / (base + 1e-9)

    table = [
        ["Metric", "Baseline", "After Det", "After Cls", "%Drop Det", "%Drop Cls"],
        ["Seg mIoU", f"{metrics_log['seg_baseline']:.2%}",
         f"{metrics_log['seg_after_det']:.2%}",
         f"{metrics_log['seg_final']:.2%}",
         f"{pct_drop(metrics_log['seg_after_det'], metrics_log['seg_baseline']):.2f} %",
         f"{pct_drop(metrics_log['seg_final'], metrics_log['seg_baseline']):.2f} %"],
        ["Det mAP", f"{metrics_log['det_baseline']:.2%}",
         f"{metrics_log['det_after_det']:.2%}",
         f"{metrics_log['det_final']:.2%}",
         "—",  # baseline 即 After Det
         f"{pct_drop(metrics_log['det_final'], metrics_log['det_after_det']):.2f} %"],
        ["Cls Top-1 Acc", f"{metrics_log['cls_baseline']:.2%}",
         f"{metrics_log['cls_after_det']:.2%}",
         f"{metrics_log['cls_final']:.2%}",
         "—", # cls_baseline is not comparable here, so we skip the first drop calc.
         f"{pct_drop(metrics_log['cls_final'], metrics_log['cls_baseline']):.2f} %"],
    ]

    print("\n" + "\n".join(["| " + " | ".join(row) + " |" for row in table]))

# 執行主程式
# 23:21
main()

使用裝置：cuda
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
✅ 已對齊 10 類，映射如下： {15: 0, 18: 1, 75: 2, 44: 3, 14: 4, 74: 5, 8: 6, 20: 7, 37: 8, 38: 9}
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
✅ 已對齊 10 類，映射如下： {15: 0, 18: 1, 75: 2, 44: 3, 14: 4, 74: 5, 8: 6, 20: 7, 37: 8, 38: 9}
訓練分類資料集樣本數：240
驗證分類資料集樣本數：60
訓練物件偵測資料集樣本數：240
驗證物件偵測資料集樣本數：60
訓練語義分割資料集樣本數：240
驗證語義分割資料集樣本數：60
🟢 Stage 1 — Segmentation


Validating CLS: 100%|██████████| 10/10 [00:00<00:00, 15.67it/s]


Validation Results -> Top-1 Acc: 0.1000 | mIoU: 0.0000 | mAP: {'map': 0.0, 'map_50': 0.0001, 'map_75': 0.0, 'map_small': 0.0, 'map_medium': 0.0, 'map_large': 0.0, 'mar_1': 0.0, 'mar_10': 0.0, 'mar_100': 0.0375, 'mar_small': 0.0, 'mar_medium': 0.0, 'mar_large': 0.0562, 'map_per_class': -1.0, 'mar_100_per_class': -1.0}
✅ 任務 'seg' 的新最佳模型已儲存至 checkpoint_seg_best.pt，分數為: 0.0000


Validating CLS: 100%|██████████| 10/10 [00:00<00:00, 16.05it/s]


Validation Results -> Top-1 Acc: 0.0833 | mIoU: 0.0520 | mAP: {'map': 0.0, 'map_50': 0.0001, 'map_75': 0.0, 'map_small': 0.0, 'map_medium': 0.0, 'map_large': 0.0, 'mar_1': 0.0, 'mar_10': 0.0, 'mar_100': 0.0417, 'mar_small': 0.0, 'mar_medium': 0.0, 'mar_large': 0.0625, 'map_per_class': -1.0, 'mar_100_per_class': -1.0}
✅ 任務 'seg' 的新最佳模型已儲存至 checkpoint_seg_best.pt，分數為: 0.0520


...

Validation Results -> Top-1 Acc: 0.4333 | mIoU: 0.0858 | mAP: {'map': 0.014, 'map_50': 0.0281, 'map_75': 0.0, 'map_small': 0.0, 'map_medium': 0.0, 'map_large': 0.0417, 'mar_1': 0.0417, 'mar_10': 0.0417, 'mar_100': 0.0417, 'mar_small': 0.0, 'mar_medium': 0.0, 'mar_large': 0.125, 'map_per_class': -1.0, 'mar_100_per_class': -1.0}


Validating CLS: 100%|██████████| 10/10 [00:00<00:00, 12.15it/s]


Validation Results -> Top-1 Acc: 0.4167 | mIoU: 0.0872 | mAP: {'map': 0.014, 'map_50': 0.0281, 'map_75': 0.0, 'map_small': 0.0, 'map_medium': 0.0, 'map_large': 0.0417, 'mar_1': 0.0417, 'mar_10': 0.0417, 'mar_100': 0.0417, 'mar_small': 0.0, 'mar_medium': 0.0, 'mar_large': 0.125, 'map_per_class': -1.0, 'mar_100_per_class': -1.0}


Validating CLS: 100%|██████████| 10/10 [00:00<00:00, 11.52it/s]

Validation Results -> Top-1 Acc: 0.4167 | mIoU: 0.0872 | mAP: {'map': 0.014, 'map_50': 0.0281, 'map_75': 0.0, 'map_small': 0.0, 'map_medium': 0.0, 'map_large': 0.0417, 'mar_1': 0.0417, 'mar_10': 0.0417, 'mar_100': 0.0417, 'mar_small': 0.0, 'mar_medium': 0.0, 'mar_large': 0.125, 'map_per_class': -1.0, 'mar_100_per_class': -1.0}

| Metric | Baseline | After Det | After Cls | %Drop Det | %Drop Cls |
| Seg mIoU | 14.93% | 13.62% | 8.72% | 8.78 % | 41.63 % |
| Det mAP | 0.00% | 0.22% | 1.40% | — | -536.36 % |
| Cls Top-1 Acc | 10.00% | 13.33% | 41.67% | — | -316.67 % |
